# Coaxial Waveguide

In [ ]:
import numpy as np
import scipy.special as sp
import numpy.testing as npt
import pymwm
from pymwm.cutoff import Cutoff

co = Cutoff(16, 8)

In [ ]:
co(('E',1,2),0.5)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()

#samples = []
#for rr in range(100):
#    sample = co.samples[rr][('E', 1, 2)]
#    samples.append(sample)

# for n in range(3):
#     for m in range(3):
#         samples = [s[('E', n, m+1)] for s in co.samples]
#         plt.plot(co.r_ratios, samples, label=f"E{n}{m+1}")

for n in range(16):
    for m in range(8):
        df = co.samples.query(f"pol=='E' and n=={n} and m=={m+1}")
        plt.plot(df['rr'], df['val'], label=f"E{n}{m+1}")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=1, fontsize=7)

plt.xlim(0,0.99)
plt.ylim(0,100)

#0.99まで有効

In [ ]:
df1 = co.samples.query(f"pol=='E' and  n == 0 and m == 1")
df2 = co.samples.query(f"pol=='E' and  n == 1 and m == 1")
display(df1)
display(df2)
display(df1['val'].to_numpy() - df2['val'].to_numpy())

In [ ]:
for m in range(7, 8):
    for n in range(1, 15):
        df1 = co.samples.query(f"pol=='E' and n=={n} and m=={m+1}")
        df2 = co.samples.query(f"pol=='E' and n=={n+1} and m=={m+1}")
        val_dif = df2['val'].to_numpy() - df1['val'].to_numpy()
        plt.plot(df1['rr'], val_dif, "--", label=f"E{n+1}{m+1}-E{n}{m+1}")
        
plt.legend()

plt.xlim(0.9,1)
plt.ylim(0,0.1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()

for n in range(16):
    for m in range(8):
        df = co.samples.query(f"pol=='M' and n=={n} and m=={m+1}")
        plt.plot(df['rr'], df['val'], label=f"M{n}{m+1}")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=1, fontsize=7)

plt.xlim(0,0.99)
plt.ylim(0,100)

#0.99まで有効

In [ ]:
import pandas as pd
df = pd.DataFrame()
for i, (r_ratio, sample) in enumerate(zip(co.r_ratios, co.samples)):
    for pol in ['E', 'M']:
        for n in range(co.num_n):
            for m in range(1, co.num_m + 1):
                df = df.append({'irr': i, 'rr': r_ratio,
                                'pol': pol, 'n': n, 'm': m, 'val': sample[(pol, n, m)]},
                               ignore_index=True)
df = df.astype({'irr': int, 'n': int, 'm': int})
df.to_hdf("cutoff.h5", "cutoff")

In [ ]:
df = df.astype({'irr': int, 'n': int, 'm': int})

In [ ]:
import pandas as pd
df=pd.read_hdf("cutoff.h5")
df.query(f"irr==1 and pol=='E' and n==1 and m==1")['val']*2

In [ ]:
df

nの差をグラフで
num_n num_m 大きめ(n16,m8)で試す

In [ ]:
import numpy as np
import pymwm
from pymwm.coax.samples import Samples
from riip import Material

wl_max = np.pi
wl_min = 0.6
air = {'RI': 1.0}
params = {'core': {'shape': 'coax', 'ri': 0.45, 'r': 0.5,
                   'fill': air},
          'clad': {'model': 'gold_dl'},
          'bounds': {'wl_max': wl_max, 'wl_min': wl_min,
                     'wl_imag': 10.0},
          'modes': {'wl_max': wl_max, 'num_n': 6, 'num_m': 2}}
#wg = pymwm.create(params)
p_fill = params['core']['fill'].copy()
p_fill['bound_check'] = False
p_clad = params['clad'].copy()
p_clad['bound_check'] = False
smp = Samples(
    params['core']['r'], Material(p_fill), Material(p_clad), params['modes'],
    params['core']['ri'])

smp.beta2_w_min(1)

In [ ]:
smp.clad(0.1)